In [1]:
import pandas as pd
import os
import numpy as np
import soundfile as sf
from IPython.display import clear_output
import pickle
import librosa
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from tensorflow.keras.models import load_model
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
# Wczytuje i łączę wszystkie dane - liczyłem embeddingi na 5 różnych kontakch kaggle.

embedding_all_people = []
embedding_all_people_postprocess = []

for i in range(0,5):
    path = '/kaggle/input/cnn-mel-embedding-test-no-postpocess/embedding_all_people_'  

    one_dataset = path + str(i) + '.pkl'
    
    with open(one_dataset, 'rb') as file:
        data = pickle.load(file)
    embedding_all_people.extend(data)

    
    path = '/kaggle/input/cnn-mel-embedding-test-postpocess/embedding_all_people_postprocessed_'  

    one_dataset = path + str(i) + '.pkl'
    
    with open(one_dataset, 'rb') as file:
        data = pickle.load(file)
    embedding_all_people_postprocess.extend(data)

In [ ]:
def create_enrollment_test(embedding_all_people):
    

    n = len(embedding_all_people[0][0][0])
    
    #Embedding enrollment ma być stworzony z 40 s nagrania

    emrollment_embedding_all_people = []  # Lista do przechowywania embeddingów enrollment dla wszystkich osób
    test_embedding_all_people = []  # Lista do przechowywania embeddingów testowych dla wszystkich osób
    
    # Pętla przetwarzająca embeddingi dla każdej osoby w embedding_all_people
    for i in range(0, len(embedding_all_people)):
        
        # Inicjalizacja zerowych embeddingów dla enrollment i testowych (o długości n) - nie mogę zadeklarować jednej wartości ponieważ 
        # w wyniku LDA mam 99 zmiennych repreneztujących embeddingi a bez LDA jest ich 128
        enrollment_embedding = np.zeros(n)
        test_embedding = np.zeros(n)
        test_embedding_one_person = []  # Lista do przechowywania testowych embeddingów dla jednej osoby
    
        # Zaokrąglenie liczby fragmentów danej osoby do najbliższej wielokrotności 10 - pozbycie się "ogonka"
        rounded_len = np.floor_divide(len(embedding_all_people[i]), 10) * 10
        
        for j in range(0, rounded_len):
            
            # Dodawanie embeddingów do enrollment dla pierwszych 40 fragmentów - enrollment ma być zbudowany z 
            # 40 s nagrania
            if j < 40:
                enrollment_embedding += embedding_all_people[i][j][0]
            
            # Po co dziesiątym fragmencie: obliczenie średniego embeddingu testowego i resetowanie `test_embedding`
            # embedding test ma mieć długośc 10 s
            elif (j + 1) % 10 == 0:
                test_embedding = test_embedding / 10
                test_embedding_one_person.append(test_embedding)  # Dodanie średniego embeddingu do listy
                test_embedding = np.zeros(n)  # Reset embeddingu testowego do kolejnych obliczeń
            
            # Dodawanie embeddingów do testowego dla kolejnych fragmentów
            else:
                test_embedding += embedding_all_people[i][j][0]
        
        # Obliczenie średniego embeddingu enrollmentowego (średnia z 40 fragmentów)
        enrollment_embedding = enrollment_embedding / 40
    
        # Dodanie średniego embeddingu enrollment i listy testowych embeddingów do głównych list
        emrollment_embedding_all_people.append(enrollment_embedding)
        test_embedding_all_people.append(test_embedding_one_person)
    
        # Wyświetlanie postępu przetwarzania dla każdej osoby
        print((i + 1) / len(embedding_all_people))
        clear_output(wait=True)

    return emrollment_embedding_all_people, test_embedding_all_people

In [19]:
enroll, test = create_enrollment_test(embedding_all_people)

1.0


In [20]:
enroll_postprocess, test_postprocess = create_enrollment_test(embedding_all_people_postprocess)

1.0


In [ ]:
# Funkcja do liczenia cross-checkingu, zwaraca ramkę danych które zawiera info o tym kto jest właścicielem enrollment, test, czy jest to próba włamania i podobieństwo kosinusowe

def cross_checking(emrollment_embedding_all_people, test_embedding_all_people):
    
    data_rows = []
    
    for i in range(0, len(emrollment_embedding_all_people)):
        embedding_enrollment = emrollment_embedding_all_people[i].reshape(1, -1)
    
        for j in range(0, len(test_embedding_all_people)):
            # Wczytanie embeddingów testowych dla danej osoby
            one_person_list = np.array(test_embedding_all_people[j])
            random_numbers = np.random.uniform(0, 1, len(one_person_list)) > 0.5
            one_person_list = one_person_list[random_numbers]
    
            for h in range(0, len(one_person_list)):
                # Wczytanie embeddingu testowego
                embedding_eval = one_person_list[h].reshape(1, -1)
                # Obliczenie podobieństwa kosinusowego
                cos_sim = cosine_similarity(embedding_enrollment, embedding_eval)[0][0]
                # Zapisanie wyników
                data_rows.append([i, j, i == j, cos_sim])
    
        clear_output(wait=True)
        print(f"Processing ID: {i}")
    
    # Tworzenie DataFrame z listy wyników
    long_data_frame = pd.DataFrame(data_rows, columns=['ID_enrollment', 'ID_test', 'genuine', 'score'])
    return long_data_frame

In [26]:
long_data_frame = cross_checking(enroll, test)

Processing ID: 49


In [27]:
long_data_frame_postprocess = cross_checking(enroll_postprocess, test_postprocess)

Processing ID: 49


In [29]:
long_data_frame.to_csv('long_data_frame.csv', index=False)
long_data_frame_postprocess.to_csv('long_data_frame_postprocess.csv', index=False)